In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def latex_label(text):
    """
    Converts regular text to a LaTeX-compatible label, handling LaTeX math mode correctly.
    """
    # Split text into math ($...$) and non-math segments
    parts = re.split(r'(\$.+?\$)', text)
    formatted_parts = []

    for part in parts:
        if part.startswith('$') and part.endswith('$'):
            # It's a math mode segment, add it as is
            formatted_parts.append(part)
        else:
            # Replace spaces with ~ and wrap non-math text with \mathrm{}
            clean_fragment = part.replace(' ', '~')
            if clean_fragment:
                latex_command = r"\mathrm{" + clean_fragment + "}"
                formatted_parts.append(f'${latex_command}$')
                # formatted_parts.append(f'${r"\mathrm{" + clean_fragment + "}"}$')

    return ''.join(formatted_parts)

In [ ]:
def plot_mpc_trajectory(ts, z_ref, z_mpc, z_true, u_mpc, N, plot_controls=True):
    """
    Plot the MPC trajectory for 2D or 3D data.
    """
    # Determine the dimension (2D or 3D)
    dim = z_true.shape[1]

    # Set up the plotting environment
    if plot_controls:
        if dim == 2:
            fig, ax = plt.subplots(1, 2, figsize=(10, 4))
        else:
            fig = plt.figure(figsize=(12, 5))
            ax = [fig.add_subplot(1, 2, 1, projection='3d'), fig.add_subplot(1, 2, 2)]
    else:
        if dim == 2:
            fig, ax = plt.subplots(1, 1, figsize=(6, 4))
            ax = [ax]
        else:
            fig = plt.figure(figsize=(6, 5))
            ax = [fig.add_subplot(1, 1, 1, projection='3d')]

    # Plot the MPC trajectories
    for t_idx in range(len(ts) - 2 * N):
        if dim == 2:
            ax[0].plot(z_mpc[t_idx, :, 0], z_mpc[t_idx, :, 1], '--*', color='k', markersize=3,
                       label=latex_label('MPC') if t_idx == 0 else None)
        else:
            ax[0].plot3D(z_mpc[t_idx, :, 0], z_mpc[t_idx, :, 1], z_mpc[t_idx, :, 2], '--*', color='k', markersize=3,
                         label=latex_label('MPC') if t_idx == 0 else None)

    # Plot the true trajectory, start point, and reference
    if dim == 2:
        ax[0].plot(z_true[:-N, 0], z_true[:-N, 1], '-o', label=latex_label('True'), markersize=3)
        ax[0].plot(z_true[0, 0], z_true[0, 1], 'ro', label=latex_label('Start'), markersize=6)
        ax[0].plot(z_ref[:-N, 0], z_ref[:-N, 1], 'y--', label=latex_label('Reference'))
        ax[0].set_xlabel(latex_label('X [m]'))
        ax[0].set_ylabel(latex_label('Y [m]'))
        ax[0].axis('equal')
    else:
        ax[0].plot3D(z_true[:-N, 0], z_true[:-N, 1], z_true[:-N, 2], '-o', label=latex_label('True'), markersize=3)
        ax[0].scatter(z_true[0, 0], z_true[0, 1], z_true[0, 2], color='r', label=latex_label('Start'), s=36)
        ax[0].plot3D(z_ref[:-N, 0], z_ref[:-N, 1], z_ref[:-N, 2], 'y--', label=latex_label('Reference'))
        ax[0].set_xlabel(latex_label('X [m]'))
        ax[0].set_ylabel(latex_label('Y [m]'))
        ax[0].set_zlabel(latex_label('Z [m]'))
        ax[0].set_box_aspect([1, 1, 1])
        ax[0].view_init(elev=115, azim=-115, roll=-25)
    ax[0].legend()

    # Plot the control inputs
    if plot_controls:
        for u_idx in range(u_mpc.shape[-1]):
            ax[1].plot(ts, u_mpc[:, 0, u_idx], label=latex_label(f'u_{u_idx+1}(t)'))
        ax[1].set_xlabel(latex_label('t [s]'))
        ax[1].set_ylabel(latex_label('U'))
        ax[1].legend()
    plt.tight_layout()

In [ ]:
def generate_ref_trajectory(T, dt, traj_type, size):
    """
    Generate a 3D reference trajectory for the system to track.
    """
    t = np.linspace(0, T, int(T/dt))
    z_ref = np.zeros((len(t), 3))

    # Note that y is up
    if traj_type == 'circle':
        z_ref[:, 0] = size * (np.cos(2 * np.pi / T * t) - 1)
        z_ref[:, 1] = size / 2 * np.ones_like(t)
        z_ref[:, 2] = size * np.sin(2 * np.pi / T * t)
    elif traj_type == 'figure_eight':
        z_ref[:, 0] = size * np.sin(2 * np.pi / T * t)
        z_ref[:, 1] = size / 2 * np.ones_like(t)
        z_ref[:, 2] = size * np.sin(4 * np.pi / T * t)
    elif traj_type == 'periodic_line':
        m = -1
        z_ref[:, 0] = size * np.sin(2 * np.pi / T * t)
        z_ref[:, 1] = np.zeros_like(t)
        z_ref[:, 2] = m * size * np.sin(2 * np.pi / T * t)
    elif traj_type == 'arc':
        m = 1
        l_trunk = 0.35
        R = l_trunk / 2
        z_ref[:, 0] = size * np.sin(2 * np.pi / T * t)
        z_ref[:, 2] = m * size * np.sin(2 * np.pi / T * t)
        z_ref[:, 1] = R - np.sqrt(R**2 - z_ref[:, 0]**2 - z_ref[:, 0]**2)
    else:
        raise ValueError('Invalid trajectory type: ' + traj_type + '. Valid options are: "circle" or "figure_eight".')
    return z_ref, t

In [ ]:
# Generate reference trajectory (check that it's the same as used in mpc_initializer_node.py)
z_ref, t = generate_ref_trajectory(10, 0.01, 'figure_eight', 0.1)